# Kaggle Basic Project : 1st
- Titanic - Machine Learning From Disaster
- Date : 2021/04/22
- Author : heo02602
- 작업범위 : 제공된 데이터 읽어오기, 기본적인 피처 엔지니어링, 로지스틱회귀 알고리즘 학습 및 예측, 분석 결과 Submit

### 1. 사용할 패키지 읽어오기
- Numpy : 숫자 계산을 지원하는 패키지
- Pandas : 테이블 형식의 데이터(테이블 데이터)를 쉽게 읽어 들일 수 있게 해주는 패키지

### 2. 데이터 목록 확인하기
- 어떤 데이터들이 제공되는지 확인한다.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


### 3. 데이터 읽어오기
- gender_submission.csv
    - submit 전용 csv 파일 샘플이다.
    - 요구하는 형식을 확인할 수 있다.
- train.csv
    - 머신러닝 학습 전용 데이터이다.
- test.csv
    - 테스트 전용 데이터이다.
    - 학습 전용 데이터를 기반으로 테스트 전용 데이터를 테스트해야 한다.

In [2]:
# 현재 작업 경로를 확인한다.
!pwd

/kaggle/working


In [3]:
# DataFrame 형식으로 데이터를 읽어온다.
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')

In [4]:
# 학습 데이터 내용 확인
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# 테스트 데이터 내용 확인
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
# 학습 데이터의 요약된 통계 정보
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### 4. Feature Engineering
- 읽어들인 데이터를 머신러닝 알고리즘에서 사용할 수 있게 변환하는 것이다.
- 기존의 데이터를 기반으로 머신러닝 알고리즘 예측할 때 유용한 새로운 피처를 만드는 것이다.

In [7]:
# 학습 데이터와 테스트 데이터를 한번에 처리할 수 있도록 세로 방향으로 결합한다.
data = pd.concat([train, test], sort=False)

In [8]:
# 각 속성의 결측값의 갯수를 계산한다.
data.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

> #### 결측값 처리 방법   
> 결측값을 그대로 다룬다.   
> 대푯값으로 결측값을 대체한다.   
> 다른 피처로 결측값을 예측해서 대체한다.   
> 결측값 여부를 기반으로 새로운 피처를 만들어서 사용한다.   

In [9]:
# 속성값 형식을 문자열에서 숫자 형식으로 변환한다. -> Sex
data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)

# 데이터 결측값을 처리한다. -> Embarked
data['Embarked'].fillna('S', inplace=True)

# 속성값 형식을 문자열에서 숫자로 변환한다. -> Embarked
data['Embarked'] = data['Embarked'].map({'S':0, 'C':1, 'Q':2}).astype(int)

# 데이터 결측값을 처리한다. -> Fare
# 평균값(mean)으로 대체한다.
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)

# 데이터 결측값을 처리한다. -> Age
# 평균값(mean)과 표준편차(std)를 기반으로 랜덤한 숫자를 생성한다.
age_avg = data['Age'].mean()
age_std = data['Age'].std()
data['Age'].fillna(np.random.randint(age_avg - age_std, age_avg + age_std), inplace=True)

In [10]:
# 결합한 학습 데이터와 테스트 데이터를 다시 분리한다.
train = data[:len(train)]
test = data[len(train):]

In [11]:
# 피처와 목적변수로 분할한다.
y_train = train['Survived']
X_train = train.drop(['Survived', 'Name', 'Ticket', 'Cabin'], axis=1)
X_test = test.drop(['Survived', 'Name', 'Ticket', 'Cabin'], axis=1)

In [12]:
X_train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,0,22.0,1,0,7.2500,0
1,2,1,1,38.0,1,0,71.2833,1
2,3,3,1,26.0,0,0,7.9250,0
3,4,1,1,35.0,1,0,53.1000,0
4,5,3,0,35.0,0,0,8.0500,0


In [13]:
y_train.head()

0    0.0
1    1.0
2    1.0
3    1.0
4    0.0
Name: Survived, dtype: float64

In [14]:
X_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,0,34.5,0,0,7.8292,2
1,893,3,1,47.0,1,0,7.0000,0
2,894,2,0,62.0,0,0,9.6875,2
3,895,3,0,27.0,0,0,8.6625,0
4,896,3,1,22.0,1,1,12.2875,0


In [15]:
# 경고메시지 무시
import warnings
warnings.filterwarnings(action='ignore')

# '로지스틱 회귀' 머신러닝 알고리즘을 사용하여 대응관계를 학습시킨다.
from sklearn.linear_model import LogisticRegression

In [16]:
# 머신러닝 알고리즘의 동작은 하이퍼파라미터 값으로 제어된다.
clf = LogisticRegression(penalty='l2', solver='sag', random_state=0)
clf.fit(X_train, y_train)

LogisticRegression(random_state=0, solver='sag')

In [17]:
# 학습 완료 후 예측
y_pred = clf.predict(X_test)

In [18]:
# 결과 저장
gender_submission['Survived'] = list(map(int, y_pred))
gender_submission.to_csv('submission.csv', index=False)